In [4]:
import os
import ssl
import httpx
from PIL import Image
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE
http_client = httpx.Client(verify=False)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY2"), http_client=http_client)

### Gerando imagem

In [11]:
nome="robo_praia"
model="dall-e-3"
prompt="A imagem mostra um jovem jogador de futebol posando com uma camisa de sua equipe. Ele está segurando a camisa, que exibe o nome ENDRICK e o número 9. O fundo da imagem contém diversos logos de patrocinadores e referências à liga, como LIGUE 1 e Emirates. O jovem usa uma jaqueta escura e uma camiseta branca por baixo, e parece sorridente e feliz por estar sendo apresentado."
size="1024x1024"
quality="hd"
style="vivid"

In [12]:
resposta = client.images.generate(
    model=model,
    prompt=prompt,
    n=1,
    size=size,
    quality=quality,
    style=style
)

In [13]:
resposta.data[0].revised_prompt

'The image showcases a youthful soccer player posing with his team jersey. He is holding the jersey bearing the name ENDRICK and number 9. The backdrop of the image is adorned with various sponsor logos and references to his league, showcasing symbols like LIGUE 1 and Emirates. The young man is dressed in a dark jacket and a white shirt underneath, bearing a broad smile, looking delighted and excited about his introduction.'

In [14]:
nome_arquivo = f"files_2/{nome}_{model}_{quality}_{style}.jpg"
image_url = resposta.data[0].url
image_data = requests.get(image_url).content
with open(nome_arquivo, "wb") as f:
    f.write(image_data)    


### Image to Text

In [9]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "Descreva a imagem fornecida"},
            {"type": "image_url", "image_url": {
                    "url": "https://tmssl.akamaized.net//images/foto/galerie/endrick-lyon-2025-2026-1767704930-186554.jpg"
                }}
            ]
        }
    ]
)

In [10]:
resposta.choices[0].message.content

'A imagem mostra um jovem jogador de futebol posando com uma camisa de sua equipe. Ele está segurando a camisa, que exibe o nome "ENDRICK" e o número "9". O fundo da imagem contém diversos logos de patrocinadores e referências à liga, como "LIGUE 1" e "Emirates". O jovem usa uma jaqueta escura e uma camiseta branca por baixo, e parece sorridente e feliz por estar sendo apresentado.'

### Extraia placa de carro

In [16]:
texto = "Qual é a placa do carro? Devolva apenas a placa!"
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                    "url": "https://www.despachantemarcelino.com.br/wp-content/uploads/2020/11/placapreta-1.jpg"
                }}
            ]
        }
    ],
    max_tokens=1000
)

In [17]:
resposta.choices[0].message.content

'GAV-1551'

### OCR

In [18]:
texto = "O que está escrito na imagem?"
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                    "url": "https://pbs.twimg.com/media/Doxk-FHW0AIW_ar.jpg:large"
                }}
            ]
        }
    ],
    max_tokens=1000
)

In [19]:
resposta.choices[0].message.content

'O texto na imagem diz:\n\n"Do meu querido Povo Brasileiro.\n\nDia 06 de outubro é o meu aniversário oficial.\n\nEspero ganhar de presente no dia 07 de outubro o voto do povo brasileiro no Haddad para Presidente. Haddad é 13, Haddad e Lula.\n\nUm grande abraço do\n\n[assinado]\n\nsem medo de ser feliz."'